In [14]:
import numpy as np
from scipy import stats

#Using Bokeh 0.12.5 and Python 3.6
from bokeh.io import output_notebook, show
from bokeh.models import Select,CustomJS
from bokeh.layouts import row,column
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

Loading BokehJS ...

In [15]:
#Loading files into Numpy arrays
#Missing values substituted by -1
data_original = np.genfromtxt('data/data_full.csv',delimiter=",",dtype=None,skip_header=1)
data_missing = np.genfromtxt('data/data_miss.csv',delimiter=",",dtype=None,skip_header=1)

#Finding Missing values and storing them in row_missing,col_missing arrays
row_missing=np.where(data_missing<0)[0]
col_missing=np.where(data_missing<0)[1]

#Subtracting the rows that have values missing
data_slice=np.delete(data_missing, row_missing, axis=0)
data_slice_missing=data_missing[row_missing,:]


In [16]:
#Estimating the value of missing data

#Approach 1: Using Mean,Median or Mode
#In this approach the mean,median and mode are calculated for the columns for which the data is missing.
#Note that data_slice doesn't have the rows that have any value missing

#Calculating Mean of an array column
def columnMean(array,col):
    return np.average(array[:,col])

#Calculating Median of an array column
def columnMedian(array,col):
    return np.median(array[:,col])

#Calculating Mode of an array column
def columnMode(array,col):
    return stats.mode(array[:,col])[0][0]



In [17]:
#Approach 2 : Using K-Nearest Neighbours

class KNN(object):
    def _init_(self):
        pass
    
    def train(self,train_matrix,train_values):
        #Just saves the values
        #train_matrix doesn't have column of the missing value, train_values is that column
    
        self.train_matrix=train_matrix
        self.train_values=train_values
        
    def predict(self,predict_row,k):
        
        #Takes only one row as input to predict the value of missing column
        #predict_row doesn't have the dimension of the missing column
        
        #Finding L2 Euclidean Distance between all rows of training set and broadcasted row whose missing value is needed
        dists=np.sqrt(np.sum(np.square(self.train_matrix-predict_row),axis=1))
        
        #Fetching the values of the k nearest neighbours corresponding to missing column
        closest_k=self.train_values[np.argsort(dists,kind='quicksort')[0:k]]
        
        #Taking the average of the values of nearest neighbours
        return np.average(closest_k)
    


In [20]:

def executeKNN(k,pred_row,pred_col):
    #X_train is the data matrix minus the row and column for which the data is missing
    #Y_train is the column for which the data is missing
    X_train=data_slice[:,[x for x in range(data_missing.shape[1]) if x!=pred_col]]
    Y_train=data_slice[:,pred_col]
    
    classifier=KNN()
    classifier.train(X_train,Y_train)
    
    pred_value=classifier.predict(data_missing[pred_row,[x for x in range(data_missing.shape[1]) if x!=pred_col]],k)
    return pred_value

    

In [21]:
#Creating datasets for plots, 1. Mean, 2. Median 3. Mode 4. 1-KNN 5.5-KNN 6. 10-KNN 7. 17-KNN 8. 20-KNN
mean_squared_errors=[]
calculated_data=[]
row_indices=[]
col_indices=[]

mean_squared_errors_dictionary={}
calculated_data_dictionary={}
row_indices_dictionary={}
col_indices_dictionary={}
x_data_dictionary={}

dictionary={}


def evaluate(col_no, estimation_type,k):
    row_nums=[]
    data_filled=data_missing;
    mean_squared=0.0
    
    for index, col in enumerate(col_missing):
        if(col==col_no):
            val=0.0
            row_no=row_missing[index]
            row_nums.append(row_no)
            if (estimation_type==1): #Mean
                val=columnMean(data_slice,col_no)
                data_filled[row_no,col_no]=val
            
            if (estimation_type==2): #Median
                val=columnMedian(data_slice,col_no)
                data_filled[row_no,col_no]=val
                #print(val)
            
            if (estimation_type==3): #Mode
                val=columnMode(data_slice,col_no)
                data_filled[row_no,col_no]=val
                #print(val)
            
            if (estimation_type==4): #KNN
                val=executeKNN(k,row_no,col_no)
                data_filled[row_no,col_no]=val
                #print(val)
                
            mean_squared=mean_squared+(data_original[row_no,col_no]-val)**2
            #print(mean_squared)
    
    
    dict_label='c'+str(col_no)+'t'+str(estimation_type)+'k'+str(k)
    
    y_axis=data_filled[:,col_no]
    calculated_data.append(np.array(y_axis))
    calculated_data_dictionary[dict_label]=np.array(y_axis)
    dictionary[dict_label+'y']=np.array(y_axis)
    
    mean_squared=mean_squared**(1/2.0)
    mean_squared_errors.append(np.array(mean_squared))
    dictionary[dict_label+'m']=np.array(mean_squared)
    
    
    
    row_indices.append(np.array(row_nums))
    row_indices_dictionary[dict_label]=np.array(row_nums)
    dictionary[dict_label+'r']=np.array(row_nums)
    
    col_indices.append(np.array(col_no))
    col_indices_dictionary[dict_label]=np.array(col_no)
    dictionary[dict_label+'c']=np.array(col_no)
    
    x_data_dictionary[dict_label]=data_original[:,col_no]
    dictionary[dict_label+'x']=np.array(data_original[:,col_no])
 
    

for x in np.unique(col_missing):
    evaluate(x,1,1)
    evaluate(x,2,1)
    evaluate(x,3,1)
    evaluate(x,4,1)
    evaluate(x,4,5)
    evaluate(x,4,10)
    evaluate(x,4,17)
    evaluate(x,4,20)



In [22]:
# Making scatter plots for original columns values vs predicted column values
#Interactions allowing user to choose the kind of estimation and column



menu=[]
menu.append(('0','Mean'))
menu.append(('1','Median'))
menu.append(('2','Mode'))
menu.append(('3','1-Nearest Neighbour'))
menu.append(('4','5-Nearest Neighbour'))
menu.append(('5','10-Nearest Neighbour'))
menu.append(('6','17-Nearest Neighbour'))
menu.append(('7','20-Nearest Neighbour'))

menu2=[]
menu2.append(('0','Milk'))
menu2.append(('1','Grocery'))
menu2.append(('2','Detergent'))
menu2.append(('3','Delicassen'))

#source=ColumnDataSource(data=dict(compType=[comp_type]))


#estimation_type_dd=Select(title="Choose Estimation type",value="0", options=menu)
#column_dd=Select(title="Choose Column",value="0", options=menu2)

#estimation_type_chosen=estimation_type_dd
#print(estimation_type_chosen.value)

comp_types=[0,1,2,3,6,8,9,10,11,14,16,17,18,19,22,24,25,26,27,30]
col_names=['Milk','Milk','Milk','Milk','Milk','Grocery','Grocery','Grocery','Grocery','Grocery','Detergents','Detergents','Detergents','Detergents','Detergents','Delicassen','Delicassen','Delicassen','Delicassen','Delicassen']
estimate_type=['Mean','Median','Mode','1-Nearest Neighbour','17-Nearest Neighbour','Mean','Median','Mode','1-Nearest Neighbour','17-Nearest Neighbour','Mean','Median','Mode','1-Nearest Neighbour','17-Nearest Neighbour','Mean','Median','Mode','1-Nearest Neighbour','17-Nearest Neighbour','Mean','Median','Mode','1-Nearest Neighbour','17-Nearest Neighbour']

figList=[]
counter=0
for comp_type in comp_types:
    figList.append(figure(plot_width=600, plot_height=350,x_range=(0, 9000), y_range=(0, 9000),title='Estimation Type:'+' '+str(estimate_type[counter])+'  '+'Column Name: '+str(col_names[counter])+'  '+'Mean Square Error:'+' '+ str(mean_squared_errors[comp_type])))
    figList[counter].circle(data_original[:,col_indices[comp_type]],calculated_data[comp_type], size=2, fill_color="blue", fill_alpha=0.5)
#Plotting  the curve
#fig =figure(plot_width=700, plot_height=450,x_range=(0, 5000), y_range=(0, 5000),title='Mean Square Error:'+ str(mean_squared_errors[comp_type]))
#sc1=fig.circle(data_original[:,col_indices[comp_type]],calculated_data[comp_type], size=2, fill_color="blue", fill_alpha=0.5)
    
#Highlighting the approximate values in yellow and actual values in red
#sc2=fig.circle(data_original[row_indices[comp_type],col_indices[comp_type]],calculated_data[comp_type][row_indices[comp_type]], size=10, fill_color="yellow",legend="Estimated Value")
#sc3=fig.circle(data_original[row_indices[comp_type],col_indices[comp_type]],data_original[row_indices[comp_type],col_indices[comp_type]], size=10, fill_color="red",legend="Actual Value")
    figList[counter].circle(data_original[row_indices[comp_type],col_indices[comp_type]],calculated_data[comp_type][row_indices[comp_type]], size=10, fill_color="yellow",legend="Estimated Value")
    figList[counter].circle(data_original[row_indices[comp_type],col_indices[comp_type]],data_original[row_indices[comp_type],col_indices[comp_type]], size=10, fill_color="red",legend="Actual Value")
    counter=counter+1


#estimation_type_dd.js_on_change('value', update_curve)
#column_dd.js_on_change('value', update_curve)

##show(row(fig,column(estimation_type_dd,column_dd)))



In [27]:
#Finding Best Estimation technique over all columns

tec_name=['Mean','Median','Mode','1-NN','5-NN','10-NN','17-NN','20-NN']
Mse_list=np.zeros(8)
colors=["#4B5C8A","red","blue","yellow","orange","magenta","firebrick","pink"]
Mse_list[0]=np.sum([mean_squared_errors[x] for x in [0,8,16,24]])
Mse_list[1]=np.sum([mean_squared_errors[x] for x in [1,9,17,25]])
Mse_list[2]=np.sum([mean_squared_errors[x] for x in [2,10,18,26]])
Mse_list[3]=np.sum([mean_squared_errors[x] for x in [3,11,19,27]])
Mse_list[4]=np.sum([mean_squared_errors[x] for x in [4,12,20,28]])
Mse_list[5]=np.sum([mean_squared_errors[x] for x in [5,13,21,29]])
Mse_list[6]=np.sum([mean_squared_errors[x] for x in [6,14,22,30]])
Mse_list[7]=np.sum([mean_squared_errors[x] for x in [7,15,23,31]])


p = figure(plot_width=900, plot_height=500,title="Mean Square Errors for different estimations")
for x in range(8):
    
    p.vbar(x=2*x, width=0.5, bottom=0,
    top=Mse_list[x], color=colors[x],legend=tec_name[x])
    
show(p)




#17 and 20 Nearest neighbour approaches seem to be working the best as the have least MSE


In [24]:
#Displaying the estimated values for column Milk
show(row(figList[:5]))

                

In [25]:
#Displaying the estimated values for column Grocery
show(row(figList[5:10]))

In [26]:
#Displaying the estimated values for column Detergents
show(row(figList[10:15]))

In [ ]:
#Displaying the estimated values for column Delicassen
show(row(figList[15:20]))